# Cross-account access to Central Infrastructure

<div class="alert alert-block alert-info">
  <b>Important:</b> Only run this notebook in AWS-led events, if your infrastructure is failing.
</div>

Copy paste the cell below in the notebooks to use cross-account invocations during an AWS workshop. You will need to pass the `boto_session` to the `sagemaker.Predictor` and to the `strands.Agent` implementation to use cross-account invocations. Examples are provided below.

In [ ]:
from utils.cross_account_utils import cross_account_session_and_sagemaker_runtime

central_account_id = input(
	"Insert here the Account ID provided by your AWS instructor:"
)

CROSS_ACCOUNT_ROLE = (
    f"arn:aws:iam::{central_account_id}:role/central-endpoint-CrossAccountRole"
)

try:
    boto_session, _ = cross_account_session_and_sagemaker_runtime(
        CROSS_ACCOUNT_ROLE
    )
    print("✓ Cross-account access configured successfully")
except Exception as e:
    print(f"⚠️ Cross-account access error: {e}")
    print("This may be expected if running outside the workshop environment")

-----

## Examples

<div class="alert alert-block alert-info">
Use the cells below to test the code, no need to copy-paste elsewhere!
</div>

#### SageMaker Predictor

In [ ]:
import sagemaker

VANILLA_ENDPOINT_NAME = "vanilla-qwen3"

predictor = sagemaker.Predictor(
    endpoint_name=VANILLA_ENDPOINT_NAME,
    sagemaker_session=sagemaker.Session(boto_session=boto_session),
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

predictor.predict(
    {
        "messages": [
            {"role": "user", "content": "What is the capital of France?"}
        ]
    }
)

#### Strands Agents

In [ ]:
from strands import Agent
from strands.models.sagemaker import SageMakerAIModel

payload_config = {"max_tokens": 1024*4, "temperature": 0, "stream": True}

vanilla_model = SageMakerAIModel(
    boto_session=boto_session,
    endpoint_config={"endpoint_name": VANILLA_ENDPOINT_NAME},
    payload_config=payload_config
)

print("=== SCHEMA-GUIDED: VANILLA MODEL ===")
vanilla_schema_agent = Agent(vanilla_model, tools=schema_guided_tools, system_prompt=schema_guided_prompt)
vanilla_schema_agent(test_query)